<a href="https://colab.research.google.com/github/waynelee9511cloud/my-colab-notebooks/blob/main/LLM_CDQC_v0_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#@title 第一步：安裝環境與指定模型

!pip -q install -U "pandas==2.2.2"
!pip -q install -U "transformers>=4.50.0" accelerate huggingface_hub openpyxl "pillow<12.0" einops safetensors tqdm trl peft datasets

import os, json, re, time
import pandas as pd
import torch
from tqdm.auto import tqdm
from huggingface_hub import login
from transformers import AutoProcessor, AutoModelForCausalLM, AutoTokenizer

# ===========================
# 1) 基本設定
# ===========================
#MODEL_ID = "BioMistral/BioMistral-7B"  # BioMistral 7B model
#MODEL_ID = "WayneLee9511/WL-BMS-SFT-1400-1020-40epochs"  # Fine tune model: BioMistral_sft_out/final
#MODEL_ID = "google/medgemma-4b-it"
MODEL_ID = "/content/drive/MyDrive/medgemma_sft_out/final_model" #medgemma fine tune
EXCEL_PATH = None  # 直接填入檔案路徑字串；若為 None 會開啟上傳視窗
OUTPUT_CSV = "validation_results.csv"
MAX_ROWS = None  # 例如 500，或設 None 全部處理
#DO_FINETUNE = True  # 若要微調改成 True（需自備/產生訓練資料）
DO_FINETUNE = False  # 若要微調改成 True（需自備/產生訓練資料）


# ---------------------------------------

# (請將這整段程式碼貼到 Cell 1，覆蓋舊的 PROMPT_CONFIG)

PROMPT_CONFIG = {
    "MH": {
        "target_column": "[Diagnosis/ Conditions]",
        "error_type": "Incorrect diagnosis name",
        "example_error": "[Mypia] should be [Myopia].",
        "example_correct": "Stage 2 Hypertension",
        "rules_text": (
            "You MUST follow these rules precisely:\n"

            # --- MH RULE 1 (已加入您截圖中的範例) ---
            "\nRULE 1: The following items are considered CORRECT. DO NOT flag them as errors. Always return 'No issues found.' for these:\n"
            "- Hypomenorrhea\n"
            "- Adjustment Insomnia\n"
            "- Pyoderma and skin bacteria infection\n"
            "- Other urticaria\n"
            "- MTX with liver function impairment\n"
            "- Herpes\n"
            "- Myopia, bilateral\n"
            "- Impacted cerumen\n"
            "- Acne, unspecified\n"
            "- Cryotherapy-complicated\n"
            "- PHOTOTHERAPY,INCLUDING  SUN-LAMP AND UV\n"
            "- Phototherapy(sun-lamp & UVL)\n"
            "- Shingles Left Shoulder\n"
            "- Increased blood liver enzyme level\n"
            "- Sprain of joints and ligaments of unspecified parts of neck, initial encounter"

            # --- MH RULE 2 (保持不變) ---
            "\nRULE 2: The following example items are INCORRECT. You MUST flag them using the 'should be' format:\n"
            "- [Mypia] should be [Myopia]\n"
            "- [Diabtes] should be [Diabetes]\n"
            "- [Tegratol Allergy] should be [Tegretol Allergy]\n"
            "- [acne valgaris] should be [acne vulgaris]\n"
            "- [Local infection of the skin and subcutaneous, unspecified] should be [Local infection of the skin and subcutaneous tissue, unspecified]\n"

            # --- MH RULE 3 (新增) ---
            "\nRULE 3: This is a SPELLING check ONLY.\n"
            "- DO NOT correct case (e.g., 'endometrial lesion' is CORRECT, do not change to 'Endometrial lesion').\n"
            "- DO NOT correct modifiers (e.g., 'Myopia, bilateral' is CORRECT, do not change to 'Myopia').\n"
            "- DO NOT correct semantics (e.g., 'Adjustment Insomnia' is CORRECT, do not change to 'Adjustment Disorder Insomnia').\n"
        )
    },
    "CM": {
        "target_column": "[Generic/Brand Name]",
        "error_type": "Incorrect drug name",
        "example_error": "[Lisnopril] should be [Lisinopril].",
        "example_correct": "Aspirin 100mg",
        "rules_text": (
            "You MUST follow these rules precisely:\n"

            # --- CM RULE 1 (您的 57 個範例) ---
            "\nRULE 1: The following items are considered CORRECT. DO NOT flag them as errors. Always return 'No issues found.' for these:\n"
            "- Aspirin 100mg\n"
            "- Paracetamol 500 mg PO\n"
            "- Tetracycline Oph Oint\n"
            "- Tetracycline Oint\n"
            "- 100 Sandimmun Neoral\n"
            "- Sinpharderm\n"
            "- Sincapin\n"
            "- oxola\n"
            "- Lorapseudo SR.F.C.\n"
            "- Losartan SR\n"
            "- Lonzumin\n"
            "- Placebo/試LY3454738\n"
            "- Nerisone Fatty Oint\n"
            "- Strocain\n"
            "- inolim\n"
            "- winiful\n"
            "- Winpred\n"
            "- Nystatin susp 100000u/mL,24mL/bot\n"
            "- Sod.fusidate\n"
            "- Clindamycin phos.\n"
            "- Tacrolimus oint 0.1%,10gm/tube\n"
            "- (Fluocinolone 0.025% + Neomycin 0.35%) ointment,5gm/tube\n"
            "- Clindamycin phos.gel 1%,15gm/tube\n"
            "- Tacrolimus oint 0.1%,10gm/tube\n"
            "- besong scalp solution 1mg(betamethasone (17-valerate)\n"
            "- Spersin oint\n"
            "- Nerisone Fatty Oint 0.1%\n"
            "- Sinpharderm A.D.E. Cream\n"
            "- Kojarclinda Cap\n"
            "- Clincin Inj\n"
            "- LOMIDINE\n"
            "- Deallergy\n"
            "- sowecaine\n"
            "- coxine\n"
            "- KOLINCIN gel 1%\n"
            "- Spersin Oint\n"
            "- AZAPRINE\n"
            "- 臨試(M22-000)Upadacitinib(RINVOQ)\n"
            "- ASTHAN\n"
            "- Conlifu\n"
            "- Ucenprubart\n"
            "- Periactin Y.K. * tab 4 mg\n"
            "- Folic acid * tab 5 mg/PTP VPP\n"
            "- Methotrexate # tab 2.5 mg\n"
            "- Dance cream 1 mg/g 10 g\n"
            "- Allegra * tab 60 mg\n"
            "- SCHEREE CREAM BOWLIN\n"
            "- O.G.S.C. TABLETS JOHNSON\n"
            "- Topsym * cream\n"
            "- MOMO CREAM 0.1%\n"
            "- ENFULON CREAM 0.1%(1MG/G)\n"
            "- SINPHARDERM CREAM 10%(100MG/G)\n"
            "- Potassium Cresolsulfonate\n"
            "- Desloratadine, DENOsin F.C\n"
            "- Dexchlorpheniramine, Dex-CTM\n"
            "- Cetirizine, Cetia\n"
            "- Norethindrone/Ethinyl Estradiol/Ferrous Fumarate (Junel Fe 1.5/30)\n"
            "- Naproxen (Naprosyn)\n"

            # --- CM RULE 2 (保持不變) ---
            "\nRULE 2: The following example items are INCORRECT. You MUST flag them using the 'should be' format:\n"
            "- [Lisnopril] should be [Lisinopril]\n"
            "- [Paracetmol] should be [Paracetamol]\n"
            "- [amoxyciilin] should be [Amoxicillin]\n"
            "- [Zrytec] should be [Zyrtec]\n"

            # --- CM RULE 3 (新增) ---
            "\nRULE 3: This is a SPELLING check ONLY.\n"
            "- DO NOT correct case (e.g., 'sowecaine' is CORRECT, do not change to 'Sowecaine').\n"
            "- DO NOT remove dosages or routes (e.g., 'Aspirin 100mg' and 'Paracetamol 500 mg PO' are CORRECT).\n"
            "- DO NOT correct brand/generic combinations (e.g., 'Naproxen (Naprosyn)' is CORRECT).\n"
        )
    },
    "AE": {
        "target_column": "[Event]",
        "error_type": "Incorrect event name",
        "example_error": "[Headchae] should be [Headache].",
        "example_correct": "Mild headache",
        "rules_text": (
            "You MUST follow these rules precisely:\n"

            # --- AE RULE 1 (保持不變) ---
            "\nRULE 1: The following items are considered CORRECT. DO NOT flag them as errors. Always return 'No issues found.' for these:\n"
            "- Mild headache\n"
            "- Vaginitis\n"
            "- Hypereosinophilia\n"
            "- Impetigo\n"
            "- Pruritus\n"
            "- Pyoderma and skin bacteria infection\n"
            "- Increased blood LDH level\n"
            "- Nausea (Grade 1)\n"

            # --- AE RULE 2 (保持不變) ---
            "\nRULE 2: The following example items are INCORRECT. You MUST flag them using the 'should be' format:\n"
            "- [Headchae] should be [Headache]\n"
            "- [Nasuea] should be [Nausea]\n"

            # --- AE RULE 3 (新增) ---
            "\nRULE 3: This is a SPELLING check ONLY.\n"
            "- DO NOT correct case (e.g., 'cellulitis' is CORRECT, do not change to 'Cellulitis').\n"
            "- DO NOT correct modifiers.\n"
            "- DO NOT correct semantics (e.g., 'Folliculitis on the left thigh' is CORRECT, do not change to 'Folliculitis').\n"
            "- DO NOT remove severity (e.g., 'Nausea (Grade 1)' is CORRECT, do not change to 'Nausea').\n"
        )
    }
}



print("Packages installed successfully. Ready to load model.")

Packages installed successfully. Ready to load model.


In [1]:
#@title 第一步：安裝環境與指定模型 (短 PROMPT_CONFIG)

!pip -q install -U "pandas==2.2.2"
!pip -q install -U "transformers>=4.50.0" accelerate huggingface_hub openpyxl "pillow<12.0" einops safetensors tqdm trl peft datasets

import os, json, re, time
import pandas as pd
import torch
from tqdm.auto import tqdm
from huggingface_hub import login
from transformers import AutoProcessor, AutoModelForCausalLM, AutoTokenizer

# ===========================
# 1) 基本設定
# ===========================
#MODEL_ID = "BioMistral/BioMistral-7B"  # BioMistral 7B model
#MODEL_ID = "WayneLee9511/WL-BMS-SFT-1400-1020-40epochs"  # Fine tune model: BioMistral_sft_out/final
#MODEL_ID = "google/medgemma-4b-it"
MODEL_ID = "/content/drive/MyDrive/medgemma_sft_out/final_model" #medgemma fine tune
EXCEL_PATH = None  # 直接填入檔案路徑字串；若為 None 會開啟上傳視窗
OUTPUT_CSV = "validation_results.csv"
MAX_ROWS = None  # 例如 500，或設 None 全部處理
#DO_FINETUNE = True  # 若要微調改成 True（需自備/產生訓練資料）
DO_FINETUNE = False  # 若要微調改成 True（需自備/產生訓練資料）


# ---------------------------------------
# (請將這整段程式碼貼到 Cell 1，覆蓋舊的 PROMPT_CONFIG)
# (此版本已移除所有 "rules_text"，因為模型已學會這些規則)

PROMPT_CONFIG = {
    "MH": {
        "target_column": "[Diagnosis/ Conditions]",
        "error_type": "Incorrect diagnosis name",
        "example_error": "[Mypia] should be [Myopia].",
        "example_correct": "Stage 2 Hypertension"
        # "rules_text": (...) 已刪除
    },
    "CM": {
        "target_column": "[Generic/Brand Name]",
        "error_type": "Incorrect drug name",
        "example_error": "[Lisnopril] should be [Lisinopril].",
        "example_correct": "Aspirin 100mg"
        # "rules_text": (...) 已刪除
    },
    "AE": {
        "target_column": "[Event]",
        "error_type": "Incorrect event name",
        "example_error": "[Headchae] should be [Headache].",
        "example_correct": "Mild headache"
        # "rules_text": (...) 已刪D除
    }
}



print("Packages installed successfully. Ready to load model.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 60.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
Packages installed successfully. Ready to load model.


In [10]:
from huggingface_hub import login
login()

In [13]:
#@title  第二步：將模型掛載於google drive方便日後快速讀取

# ===========================
# 3) 載入模型與處理器（bf16 + 自動放到 GPU）
# - 使用 dtype 取代舊參數 torch_dtype
# ===========================
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['HF_HOME'] = '/content/drive/MyDrive/LLMcache6'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/LLMcache6'


#MODEL_ID = "BioMistral/BioMistral-7B"
#MODEL_ID = "WayneLee9511/WL-BMS-SFT-1400-1020-40epochs"
#MODEL_ID = "google/medgemma-4b-it"
MODEL_ID = "/content/drive/MyDrive/medgemma_sft_out/final_model" #medgemma fine tune
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

print("⏳ 正在載入模型...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch_dtype,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

processor = tokenizer
model.eval() ;
print("✅ 模型與 Tokenizer 已成功載入！")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ 正在載入模型...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 模型與 Tokenizer 已成功載入！


In [16]:
#@title  第三步：模型會跳出上傳按鍵，於讀Excel後產結果
# (此版本已包含所有修正：BATCH=8, rules_text, validate_many索引, Regex, 和 pd.merge KeyError)

from google.colab import files
from tqdm.auto import tqdm # <-- 確保 tqdm 被導入
import re # <-- 確保 re 被導入

print("請上傳 xlsx 檔案...")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("未上傳任何檔案")
EXCEL_PATH = next(iter(uploaded))
print("已選擇檔案：", EXCEL_PATH)

sheets = pd.read_excel(EXCEL_PATH, sheet_name=None, engine="openpyxl")
if MAX_ROWS is not None:
    for k in list(sheets.keys()):
        sheets[k] = sheets[k].head(MAX_ROWS)
total_rows = sum(len(df) for df in sheets.values())
print(f"已讀取工作表數：{len(sheets)}，總筆數：{total_rows}")


def row_to_yaml(row_dict):

    try:
        import yaml

        for k, v in row_dict.items():

            if "Timestamp" in str(type(v)):
                row_dict[k] = str(v)

    except Exception:
        return "\n".join([f"{k}: {v}" for k, v in row_dict.items()])
    return yaml.safe_dump(row_dict, allow_unicode=True, sort_keys=False)

def build_messages(row_index, row_dict, dynamic_system_prompt):
    # 這是原始的正確邏B輯：只過濾存在於 prompt 中的欄位 (即 target_column)
    filtered_dict = {key: value for key, value in row_dict.items() if key in dynamic_system_prompt}

    if not filtered_dict:
        return None

    yaml_text = row_to_yaml(filtered_dict)

    # --- 修正：移除 \\n 以支援 MedGemma ---
    user_text = f"The following is a single data entry（YAML）：\n{yaml_text}\n Please review it according to the guidelines and return valid JSON."
    messages = [
        {"role": "user", "content": f"{dynamic_system_prompt}\n{user_text}"}
    ]
    return messages

@torch.inference_mode()
def validate_one(row_index, row_dict): # (此函數目前未被 validate_many 使用)

    messages = build_messages(row_index, row_dict)

    prompt = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    inputs = processor(
        text=prompt,
        return_tensors="pt",
        padding=True
    ).to(model.device, dtype=torch_dtype)

    input_len = inputs["input_ids"].shape[-1]
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=512, # max_new_tokens 可從 128 增加到 512
        do_sample=False,
        temperature=None,
        repetition_penalty=1.05,
        use_cache=True,
    )
    text = processor.batch_decode(gen_ids[:, input_len:], skip_special_tokens=True)[0]


    jtxt = extract_json_str(text)
    parsed = None
    is_valid_json = False
    try:
        parsed = json.loads(jtxt) if jtxt else None
        is_valid_json = True if parsed else False
    except Exception:
        parsed = None
        is_valid_json = False

    return {
        "raw": text,
        "json_str": jtxt,
        "json_obj": parsed,
        "is_valid_json": is_valid_json
    }


def validate_many(start_index, rows, dynamic_system_prompt):
    prompts = []
    for row_index, row_dict in enumerate(rows):
        messages = build_messages(row_index, row_dict, dynamic_system_prompt)
        if messages is None:
            prompts.append(None)
            continue

        if "medgemma" in MODEL_ID.lower():
            instruction = messages[0]['content']
            prompt = f"Gemma instruction: {instruction}\n\nGemma response:\n"
        else:
            prompt = processor.apply_chat_template(
              messages,
              add_generation_prompt=True,
              tokenize=False
        )

        prompts.append(prompt)

    # --- 【!!! 關鍵修正：索引對齊 !!!】 ---
    # 將 None 轉為空字串，以保持索引對齊，防止索引錯位
    processed_prompts = [p if p is not None else "" for p in prompts]
    if all(p == "" for p in processed_prompts):
        return [{"raw": "Skipped: No target column found.", "summary": "Skipped (No target column)"} for _ in prompts]

    inputs = processor(text=processed_prompts, return_tensors="pt", padding=True)
    # --- 修正結束 ---

    for k in inputs.keys():
        if hasattr(inputs[k], "to"):
            inputs[k] = inputs[k].to(model.device)

    input_len = inputs["input_ids"].shape[-1]
    gen_ids = model.generate(**inputs, max_new_tokens=512, do_sample=False, repetition_penalty=1.05, use_cache=True)
    texts = processor.batch_decode(gen_ids[:, input_len:], skip_special_tokens=True)


    outs = []

    summary_regex = re.compile(r'\"summary\":\s*\"(.*?)\"', re.DOTALL)
    reason_regex = re.compile(r'\"reason\":\s*\"(.*?)\"', re.DOTALL)

    # --- 修正：使用 i 遍歷，確保索引對齊 ---
    for i in range(len(prompts)):
        if prompts[i] is None:
            # 這是我們一開始就跳過的行
            outs.append({
                "raw": "Skipped: No target column found.",
                "summary": "Skipped (No target column)"
            })
            continue

        # 從 texts 中獲取對應的輸出
        # (texts 和 prompts 的長度現在是一致的)
        t = texts[i]
        summary_text = ""
        summaries_found = summary_regex.findall(t)

        if summaries_found:
            summary_text = summaries_found[0]
        else:
            reasons_found = reason_regex.findall(t)
            if reasons_found:
                summary_text = "; ".join(reasons_found)
            else:
                summary_text = f"JSON parse error: {t}"

        outs.append({
            "raw": t,
            "summary": summary_text
        })
    return outs


def _safe_name(name):
    return re.sub(r"[^A-Za-z0-9_.-]+", "_", str(name))

# ===========================
# 【!!! 關鍵修正 1 !!!】
# 降低 BATCH 大小以防止模型「看錯行」
# ===========================
BATCH = 64  # <-- 可選 8, 16, 32, 64 (AI一次看幾行 影響速度)***********************************

all_records = []
per_sheet_records = {}

for sheet_name, df in sheets.items():
    print(f"處理工作表：{sheet_name}，筆數：{len(df)}")


    config = PROMPT_CONFIG.get(sheet_name)
    if not config:
        print(f"警告：在 PROMPT_CONFIG 中找不到工作表 '{sheet_name}' 的設定，將略過此工作表。")
        continue

    # ===========================
    # 【!!! 關鍵修正 2 !!!】
    # 修正 dynamic_system_prompt 的建立邏輯
    # ===========================
    rules = config.get("rules_text", "")
    if rules:
        rules = "\n" + rules + "\n"

    dynamic_system_prompt = (
        f"You are a specialized tool for identifying spelling errors in the '{config['target_column']}' field.",
        f"Your ONLY task is to check for spelling errors in the value of the '{config['target_column']}' field.",
        "You MUST IGNORE all other fields.",

        f"{rules}",  # 插入所有規則 (RULE 1, 2, 3)

        "\n--- JSON Format Rules & Examples ---",
        f"If you find a spelling error, your reason MUST be '{config['error_type']}: {config['example_error']}'.",
        "If there are no spelling errors, your summary must be 'No issues found.'.",
        "Your entire response MUST be a single, valid JSON object and nothing else.",

        "\n--- Example 1: Data with a spelling error ---\n"
        f"Example Input Data:\n"
        f"{config['target_column']}: {config['example_error'].split(' should be ')[0][1:]}\n"
        "Example JSON Output:\n"
        '{\n'
        '  "row_index": 0,\n'
        '  "issues": [\n'
        '    {\n'
        f'      "field": "{config["target_column"]}",\n'
        f'      "reason": "{config["error_type"]}: {config["example_error"]}",\n'
        '      "severity": "error"\n'
        '    }\n'
        '  ],\n'
        f'  "summary": "{config["error_type"]}: {config["example_error"]}"\n'
        '}',

        "\n--- Example 2: Data with no spelling issues ---\n"
        f"Example Input Data:\n"
        f"{config['target_column']}: {config['example_correct']}\n"
        "Example JSON Output:\n"
        '{\n'
        '  "row_index": 1,\n'
        '  "issues": [],\n'
        '  "summary": "No issues found."\n'
        '}'
    )

    dynamic_system_prompt = "\n".join(dynamic_system_prompt)

    sheet_recs = []
    rows = df.to_dict('records')

    # 使用 tqdm 顯示進度
    for s in tqdm(range(0, len(rows), BATCH), desc=f"Processing {sheet_name}"):
        batch_rows = rows[s:s+BATCH]
        outs = validate_many(s, batch_rows, dynamic_system_prompt)

        for j, out in enumerate(outs):
            row_idx = s + j
            summary = out.get("summary", "Error: Summary not found.")
            raw_output = out.get("raw", "Error: Raw output not found.")

            # ===========================
            # 【!!! 關鍵修正 3 !!!】
            # 更新後處理 (Regex) 邏輯以過濾偽陽性 (適用於所有 sheet)
            # ===========================
            if "should be" in summary and "Incorrect" in summary:
                # 抓取 ": (原始詞) should be (修正詞)."
                match = re.search(r':\s*(.+?)\s+should be\s+(.+?)\.?$', summary)
                if match:
                    original_word = match.group(1).strip()
                    corrected_word = match.group(2).strip().strip('.')

                    # 過濾「自己=自己」和「大小寫」錯誤
                    if original_word.lower() == corrected_word.lower():
                        summary = "No issues found."

            rec = {
                "sheet_name": sheet_name,
                "row_index": int(row_idx),
                "AI_summary": summary,  # 欄位名稱保持不變
            }
            sheet_recs.append(rec)
            all_records.append(rec)

    per_sheet_records[sheet_name] = sheet_recs


overview_df = pd.DataFrame(all_records)
overview_df.to_csv("validation_results__overview.csv", index=False, encoding="utf-8-sig")
print("已輸出：validation_results__overview.csv")

for sheet_name, df in sheets.items():
    if sheet_name in per_sheet_records:
        result_df = pd.DataFrame(per_sheet_records[sheet_name])

        df_with_index = df.reset_index().rename(columns={'index': 'row_index'})

        # ===========================
        # 【!!! 關鍵修正 4 (KeyError) !!!】
        # 僅在 'row_index' 上合併
        # ===========================
        merged = pd.merge(df_with_index, result_df, on="row_index", how="left")

        out_name = f"validation_results__{_safe_name(sheet_name)}.csv"
        merged.to_csv(out_name, index=False, encoding="utf-8-sig")
        print(f"已輸出：{out_name}")
    else:
         print(f"⚠️ 略過工作表 '{sheet_name}' (在 per_sheet_records 中找不到結果)")


# ===========================
# （選用）微調：SFT（文字-only），預設關閉
# ===========================
if DO_FINETUNE:
    from datasets import Dataset
    from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
    from peft import LoraConfig, get_peft_model

    # (警告：此處的程式碼仍然有問題，請保持 DO_FINETUNE = False)

    valid_records = [r for r in all_records if r.get("is_valid_json", False)]

    train_samples = []
    for r in valid_records:
        row_idx = r["row_index"]
        sheet_name = r["sheet_name"]

        original_df = sheets.get(sheet_name)
        if original_df is None or row_idx >= len(original_df):
            continue

        row_dict = {k: (None if pd.isna(v) else v) for k, v in original_df.iloc[row_idx].to_dict().items()}
        user_yaml = row_to_yaml(row_dict)
        user_text = f"Based on the following data（YAML）：\n{user_yaml}\n Please review according to the guidelines and return valid JSON."
        assistant_text = r["medgemma_json"]
        messages = [
            {"role": "user", "content": user_text},
            {"role": "assistant", "content": assistant_text},
        ]
        train_samples.append({"messages": messages})

    if len(train_samples) < 10:
        print(f"可用有效訓練樣本太少 ({len(train_samples)} < 10)，略過微調。請自行提供標註資料或使用能產出更多有效 JSON 的模型，並重試。")
    else:
        print(f"找到 {len(train_samples)} 個有效訓練樣本，開始微調。")
        raw_ds = Dataset.from_list(train_samples)

        def encode_fn(ex):
            msgs = ex["messages"]
            prompt = processor.apply_chat_template(
                msgs, add_generation_prompt=False, tokenize=False
            )
            tokenized = processor(text=prompt, return_tensors=None)
            return {"input_ids": tokenized["input_ids"], "labels": tokenized["input_ids"]}

        ds = raw_ds.map(encode_fn, remove_columns=raw_ds.column_names, batched=False)

        try:
            peft_config = LoraConfig(
                r=8, lora_alpha=16, lora_dropout=0.05,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                task_type="CAUSAL_LM",
            )
            model.enable_input_require_grads()
            model = get_peft_model(model, peft_config)
        except Exception as e:
            print(f"LoRA 套用失敗，改用全參數微調（小批次）：{e}")

請上傳 xlsx 檔案...


Saving 825AD_CM_20251020_dry run.xlsx to 825AD_CM_20251020_dry run.xlsx
已選擇檔案： 825AD_CM_20251020_dry run.xlsx
已讀取工作表數：1，總筆數：911
處理工作表：CM，筆數：911


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processing CM:   0%|          | 0/15 [00:00<?, ?it/s]

已輸出：validation_results__overview.csv
已輸出：validation_results__CM.csv


In [3]:
#@title 步驟一：從 PROMPT_CONFIG 建立微調「教材」

import re
import json
from datasets import Dataset

# 1. 這是我們給模型的「基礎指令」(已移除所有範例和規則)
#    我們將在微調時教會它規則。
BASE_INSTRUCTION_TEMPLATE = """You are a specialized tool for identifying spelling errors in the '{target_column}' field.
Your ONLY task is to check for spelling errors in the value of the '{target_column}' field.
You MUST IGNORE all other fields.
If you find a spelling error, your reason MUST be '{error_type}: [{original}] should be [{corrected}].'.
If there are no spelling errors, your summary must be 'No issues found.'.
Your entire response MUST be a single, valid JSON object and nothing else."""

training_data = []
rule1_regex = re.compile(r"-\s*(.+)")
rule2_regex = re.compile(r"-\s*\[(.+?)\]\s*should be\s*\[(.+?)\]")

print(f"正在從 PROMPT_CONFIG 產生訓練教材...")

for sheet, config in PROMPT_CONFIG.items():
    target_col = config['target_column']
    error_type = config['error_type']
    rules = config.get('rules_text', "")

    # 建立基礎指令
    # (我們從 example_error 中取一個[錯]和[對]的詞作為模板)
    example_err_match = re.search(r"\[(.+?)\]\s*should be\s*\[(.+?)\]", config['example_error'])
    original_tpl = example_err_match.group(1)
    corrected_tpl = example_err_match.group(2).strip('.')

    base_instruction = BASE_INSTRUCTION_TEMPLATE.format(
        target_column=target_col,
        error_type=error_type,
        original=original_tpl,
        corrected=corrected_tpl
    )

    # 解析 RULE 1 (正確範例)
    rule1_text = rules.split("RULE 1:")[1].split("RULE 2:")[0]
    for line in rule1_text.split('\\n'):
        match = rule1_regex.search(line)
        if match:
            correct_text = match.group(1).strip()
            if not correct_text:
                continue

            # 這是「問題」
            instruction = f"{base_instruction}\n\n--- Data Entry ---\n{target_col}: {correct_text}"

            # 這是「標準答案」
            response = json.dumps({
                "row_index": 0,
                "issues": [],
                "summary": "No issues found."
            }, indent=2)

            training_data.append({"instruction": instruction, "response": response})

    # 解析 RULE 2 (錯誤範例)
    rule2_text = rules.split("RULE 2:")[1].split("RULE 3:")[0]
    for line in rule2_text.split('\\n'):
        match = rule2_regex.search(line)
        if match:
            original_text = match.group(1).strip()
            corrected_text = match.group(2).strip()
            if not original_text:
                continue

            # 這是「問題」
            instruction = f"{base_instruction}\n\n--- Data Entry ---\n{target_col}: {original_text}"

            # 這是「標準答案」
            reason = f"{error_type}: [{original_text}] should be [{corrected_text}]."
            response = json.dumps({
                "row_index": 0,
                "issues": [
                    {
                        "field": target_col,
                        "reason": reason,
                        "severity": "error"
                    }
                ],
                "summary": reason
            }, indent=2)

            training_data.append({"instruction": instruction, "response": response})

print(f"✅ 成功產生 {len(training_data)} 筆訓練教材。")

# 將教材轉換為 Hugging Face Dataset 物件
# (我們只用 1% 當作驗證集，因為資料量很少)
raw_dataset = Dataset.from_list(training_data).train_test_split(test_size=0.01)
print(raw_dataset)

正在從 PROMPT_CONFIG 產生訓練教材...
✅ 成功產生 6 筆訓練教材。
DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 5
    })
    test: Dataset({
        features: ['instruction', 'response'],
        num_rows: 1
    })
})


In [4]:
#@title 步驟二：執行微調 (SFT)

from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# --- 1. 重新載入基礎模型和 Tokenizer ---
# (確保 Cell 3 已執行，MODEL_ID 和 torch_dtype 已定義)
print(f"正在重新載入基礎模型 {MODEL_ID} 以進行訓練...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch_dtype,
    device_map="auto",

)
# 重新載入 tokenizer (以防萬一)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # SFT 建議

# --- 2. 設定 LoRA (PEFT) ---
print("正在設定 LoRA...")
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,  # r 設為 16 (原為 8)
    lora_alpha=32, # lora_alpha 設為 32 (原為 16)
    lora_dropout=0.05,
    # MedGemma (Gemma 2) 的模組名稱
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# --- 3. 修正後的「教材」編碼函數 ---
def encode_fn(examples):
    instructions = examples["instruction"]
    responses = examples["response"]

    # 我們必須手動建立 MedGemma 的 Prompt 格式
    prompts = [
        f"Gemma instruction: {inst}\n\nGemma response: {resp}{tokenizer.eos_token}"
        for inst, resp in zip(instructions, responses)
    ]

    # Tokenize 完整的 prompt
    tokenized_full = tokenizer(prompts, truncation=True, max_length=1024, padding=False)

    # 另外 Tokenize "instruction" 部分，以便稍後 Mask 掉
    prompt_instructions_only = [f"Gemma instruction: {inst}\n\nGemma response: " for inst in instructions]
    tokenized_instructions = tokenizer(prompt_instructions_only, truncation=True, max_length=1024, padding=False)

    labels = []
    for i in range(len(tokenized_full["input_ids"])):
        instruction_len = len(tokenized_instructions["input_ids"][i])
        full_ids = tokenized_full["input_ids"][i]

        # 建立 labels，將 "instruction" 部分設為 -100 (不計算 loss)
        label = list(full_ids) # 複製
        label[:instruction_len] = [-100] * instruction_len
        labels.append(label)

    return {
        "input_ids": tokenized_full["input_ids"],
        "attention_mask": tokenized_full["attention_mask"],
        "labels": labels
    }

print("正在對教材進行編碼...")
tokenized_dataset = raw_dataset.map(
    encode_fn,
    batched=True,
    remove_columns=raw_dataset["train"].column_names
)

# --- 4. 設定訓練參數 ---
# (使用 DataCollatorForLanguageModeling，因為我們已手動處理 labels)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

OUTPUT_DIR = "/content/drive/MyDrive/medgemma_sft_out"
print(f"訓練輸出目錄：{OUTPUT_DIR}")

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,  # 由於資料量很小，batch 設 1
    gradient_accumulation_steps=4,  # (1*4 = 總 batch 4)
    learning_rate=2e-5,
    num_train_epochs=20,  # <-- 訓練 5 個 epoch，因為教材很少
    logging_steps=5,
    save_strategy="epoch", # 每個 epoch 儲存一次
    bf16=torch.cuda.is_available(),
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# --- 5. 開始訓練 ---
print("--- 🚀 開始微調訓練 ---")
trainer.train()
print("--- ✅ 微調完成 ---")

# --- 6. 儲存最終模型 ---
FINAL_MODEL_PATH = f"{OUTPUT_DIR}/final_model"
trainer.save_model(FINAL_MODEL_PATH)
tokenizer.save_pretrained(FINAL_MODEL_PATH)
print(f"✅ 最終模型已儲存至：{FINAL_MODEL_PATH}")

正在重新載入基礎模型 google/medgemma-4b-it 以進行訓練...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

正在設定 LoRA...
trainable params: 32,788,480 || all params: 4,332,867,952 || trainable%: 0.7567
正在對教材進行編碼...


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/tmp/ipython-input-2195670302.py:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


訓練輸出目錄：/content/drive/MyDrive/medgemma_sft_out
--- 🚀 開始微調訓練 ---


Step,Training Loss
5,2.340100
10,2.145200
15,1.912300
20,1.696700
25,1.422000
30,1.256200
35,1.282600
40,1.264300


--- ✅ 微調完成 ---
✅ 最終模型已儲存至：/content/drive/MyDrive/medgemma_sft_out/final_model
